# Preparação de Dados (N1)

## 🗂️ [ Projeto ]

**Nome do Projeto:** [nome do projeto]  
**Dataset:** [Nome do dataset ou fonte de dados]  
**Objetivo Geral:** [Breve Explicação do propósito — ex.: “Analisar o comportamento de churn e identificar fatores de retenção de clientes.”]  

---

## 📋 Descrição Geral do Tratamento

Nesta etapa serão realizados os principais **processos de limpeza, transformação e padronização dos dados**.  
O foco é garantir que o dataset esteja **consistente, estruturado e pronto para análises posteriores**.

**Tarefas previstas:**  
- Leitura e diagnóstico inicial do dataset.  
- Tratamento de valores nulos, duplicados e outliers.  
- Conversão de tipos e padronização de nomes de colunas.  
- Codificação de variáveis categóricas e normalização de numéricas (quando necessário).  
- Geração de artefatos intermediários e relatório de qualidade dos dados.

## 🔧 Configuração do Projeto

In [9]:
# -*- coding: utf-8 -*-
# Configurações do projeto
from pathlib import Path
import numpy as np
import pandas as pd
import logging
import sys
import json
from typing import Dict, Any, Optional

# ---- Util: busca "para cima" até achar o caminho relativo solicitado ----
def find_upwards(relative_path: str, start: Optional[Path] = None) -> Optional[Path]:
    """
    Procura por 'relative_path' subindo diretórios a partir de 'start' (ou CWD).
    Retorna o Path encontrado ou None se não existir.
    """
    start = start or Path.cwd()
    rel_parts = Path(relative_path).parts
    for base in (start, *start.parents):
        candidate = base.joinpath(*rel_parts)
        if candidate.exists():
            return candidate
    return None

# ---- Carregamento de configuração (defaults + overrides locais) ----
def load_config(base_rel="config/defaults.json", local_rel="config/local.json") -> Dict[str, Any]:
    """
    Carrega as configurações do projeto:
    - defaults.json (obrigatório na raiz do projeto)
    - local.json (opcional; sobrescreve chaves do defaults)
    A busca é feita subindo diretórios a partir do CWD.
    """
    base_p = find_upwards(base_rel)
    local_p = find_upwards(local_rel)

    if base_p is None:
        print(f"[AVISO] Arquivo {base_rel} não encontrado a partir de {Path.cwd()}. "
              f"Crie {base_rel} na raiz do projeto.")
        return {}

    config = json.loads(base_p.read_text(encoding="utf-8"))

    if local_p and local_p.exists():
        local_cfg = json.loads(local_p.read_text(encoding="utf-8"))
        config.update(local_cfg)

    print(f"[INFO] Config carregada de: {base_p}")
    if local_p and local_p.exists():
        print(f"[INFO] Overrides locais: {local_p}")

    return config

# ---- Carrega configurações ----
config: Dict[str, Any] = load_config()

# ---- Paths (alinhados à estrutura do template) ----
PROJECT_ROOT = Path.cwd()
DATA_DIR = PROJECT_ROOT / "data"
RAW_DIR = DATA_DIR / "raw"
INTERIM_DIR = DATA_DIR / "interim"
PROCESSED_DIR = DATA_DIR / "processed"
REPORTS_DIR = PROJECT_ROOT / "reports"
ARTIFACTS_DIR = PROJECT_ROOT / "artifacts"
PRINTS_DIR = PROJECT_ROOT / "prints"
DASHBOARDS_DIR = PROJECT_ROOT / "dashboards"

for d in [RAW_DIR, INTERIM_DIR, PROCESSED_DIR, REPORTS_DIR, ARTIFACTS_DIR, PRINTS_DIR, DASHBOARDS_DIR]:
    d.mkdir(parents=True, exist_ok=True)

# ---- Seed & display ----
# Define uma semente global (reprodutibilidade entre execuções) e configura parâmetros de exibição do pandas para facilitar a leitura de tabelas extensas.
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
pd.set_option('display.max_columns', 200)
pd.set_option('display.width', 120)

# ---- Logging simples ----
LOG_FILE = REPORTS_DIR / 'data_preparation.log'
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s | %(levelname)s | %(message)s',
    handlers=[
        logging.StreamHandler(sys.stdout),
        logging.FileHandler(LOG_FILE, encoding='utf-8')
    ]
)
logger = logging.getLogger(__name__)

# ---- Nome dos arquivos principais ----
INPUT_FILE = RAW_DIR / 'dataset.csv'
OUTPUT_INTERIM = INTERIM_DIR / 'dataset_interim.csv'
OUTPUT_PROCESSED = PROCESSED_DIR / 'dataset_processed.csv'

logger.info('Project configuration loaded.')


[INFO] Config carregada de: C:\Users\fabio\Projetos DEV\data projects\data-project-template-base\config\defaults.json
[INFO] Overrides locais: C:\Users\fabio\Projetos DEV\data projects\data-project-template-base\config\local.json
2025-10-27 05:39:45,652 | INFO | Project configuration loaded.


## 🧩 Funções Utilitárias

Funções auxiliares para carregamento, inspeção, redução de memória, tipagem, faltantes e outliers.  

In [10]:
# Utilidades gerais de dados
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler, MinMaxScaler

def load_csv(filepath: Path, **read_kwargs) -> pd.DataFrame:
    """Carrega CSV com opções explícitas. 
    - read_kwargs pode receber sep, encoding, dtype, na_values, etc.
    """
    logger.info(f'Loading CSV: {filepath}')
    df = pd.read_csv(filepath, **read_kwargs)
    return df

def save_parquet(df: pd.DataFrame, filepath: Path) -> None:
    """Salva DataFrame em parquet com feedback."""
    filepath.parent.mkdir(parents=True, exist_ok=True)
    df.to_parquet(filepath, index=False)
    logger.info(f'Saved parquet to: {filepath}')

def basic_overview(df: pd.DataFrame) -> Dict[str, Any]:
    """Retorna resumo rápido para diagnóstico inicial."""
    info = {
        "shape": df.shape,
        "columns": df.columns.tolist(),
        "dtypes": {c: str(t) for c, t in df.dtypes.items()},
        "memory_mb": float(df.memory_usage(deep=True).sum() / (1024**2)),
        "na_counts": df.isna().sum().to_dict()
    }
    return info

def reduce_memory_usage(df: pd.DataFrame) -> pd.DataFrame:
    """Tenta reduzir memória para colunas numéricas inteiras e floats."""
    start_mem = df.memory_usage(deep=True).sum() / 1024**2
    for col in df.select_dtypes(include=['int64', 'int32', 'int16']).columns:
        df[col] = pd.to_numeric(df[col], downcast='integer')
    for col in df.select_dtypes(include=['float64', 'float32']).columns:
        df[col] = pd.to_numeric(df[col], downcast='float')
    end_mem = df.memory_usage(deep=True).sum() / 1024**2
    logger.info(f'Memory reduced: {start_mem:.2f}MB -> {end_mem:.2f}MB')
    return df

def infer_numeric_like(df: pd.DataFrame, columns: Optional[List[str]] = None) -> pd.DataFrame:
    """Converte colunas com números em string (ex.: '1,234' / '1.234,56') para numéricas quando possível."""
    target_cols = columns or df.select_dtypes(include=['object']).columns.tolist()
    for col in target_cols:
        # remove símbolos comuns e normaliza separadores
        series = df[col].astype(str).str.replace(r'[\s\$%]', '', regex=True)
        series = series.str.replace('.', '', regex=False).str.replace(',', '.', regex=False)
        try:
            converted = pd.to_numeric(series, errors='raise')
            # Se a conversão tiver sucesso e gerar variação, assume cast
            if converted.notna().sum() > 0 and converted.dtype.kind in 'fi':
                df[col] = converted
        except Exception:
            pass
    return df

def strip_whitespace(df: pd.DataFrame) -> pd.DataFrame:
    """Remove espaços extras em colunas de texto."""
    for col in df.select_dtypes(include=['object']).columns:
        df[col] = df[col].astype(str).str.strip()
    return df

def missing_report(df: pd.DataFrame) -> pd.DataFrame:
    """Gera relatório de faltantes em ordem decrescente."""
    rep = df.isna().mean().sort_values(ascending=False).rename('missing_rate').to_frame()
    rep['missing_count'] = df.isna().sum()
    return rep

def simple_impute(df: pd.DataFrame) -> pd.DataFrame:
    """Imputação simples: média (num), moda (cat)."""
    num_cols = df.select_dtypes(include=['number']).columns
    cat_cols = df.select_dtypes(exclude=['number']).columns
    for c in num_cols:
        if df[c].isna().any():
            df[c] = df[c].fillna(df[c].median())
    for c in cat_cols:
        if df[c].isna().any():
            df[c] = df[c].fillna(df[c].mode().iloc[0])
    return df

def detect_outliers_iqr(df: pd.DataFrame, cols: Optional[List[str]] = None) -> pd.DataFrame:
    """Marca outliers via IQR adicionando colunas booleanas *_is_outlier."""
    cols = cols or df.select_dtypes(include=['number']).columns.tolist()
    for c in cols:
        q1 = df[c].quantile(0.25)
        q3 = df[c].quantile(0.75)
        iqr = q3 - q1
        lower = q1 - 1.5 * iqr
        upper = q3 + 1.5 * iqr
        df[f'{c}_is_outlier'] = (df[c] < lower) | (df[c] > upper)
    return df

def detect_outliers_zscore(df: pd.DataFrame, threshold: float = 3.0, cols: Optional[List[str]] = None) -> pd.DataFrame:
    """Marca outliers via Z-score."""
    cols = cols or df.select_dtypes(include=['number']).columns.tolist()
    for c in cols:
        mu, sigma = df[c].mean(), df[c].std(ddof=0)
        if sigma == 0:
            df[f'{c}_is_outlier'] = False
        else:
            z = (df[c] - mu) / sigma
            df[f'{c}_is_outlier'] = z.abs() > threshold
    return df

def deduplicate_rows(df: pd.DataFrame) -> pd.DataFrame:
    """Remove duplicidades exatas e registra contagem."""
    before = len(df)
    df = df.drop_duplicates()
    after = len(df)
    logger.info(f'Removed duplicates: {before - after}')
    return df

def encode_categories(df: pd.DataFrame, encoding: str = 'onehot') -> Tuple[pd.DataFrame, Dict[str, Any]]:
    """Codifica colunas categóricas e retorna df transformado + metadados da transformação."""
    cat_cols = df.select_dtypes(include=['object', 'category']).columns.tolist()
    meta: Dict[str, Any] = {"categorical_columns": cat_cols, "encoding": encoding}
    if not cat_cols:
        return df, meta

    if encoding == 'onehot':
        encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
        arr = encoder.fit_transform(df[cat_cols])
        encoded = pd.DataFrame(arr, columns=encoder.get_feature_names_out(cat_cols), index=df.index)
        df = pd.concat([df.drop(columns=cat_cols), encoded], axis=1)
        meta['categories_'] = {c: cats.tolist() for c, cats in zip(cat_cols, encoder.categories_)}
    elif encoding == 'ordinal':
        encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
        df[cat_cols] = encoder.fit_transform(df[cat_cols])
        meta['categories_'] = {c: list(map(str, cats)) for c, cats in zip(cat_cols, encoder.categories_)}
    else:
        raise ValueError("Unsupported encoding type.")
    return df, meta

def scale_numeric(df: pd.DataFrame, method: str = 'standard') -> Tuple[pd.DataFrame, Dict[str, Any]]:
    """Escala colunas numéricas (opcional)."""
    num_cols = df.select_dtypes(include=['number']).columns.tolist()
    meta: Dict[str, Any] = {"numeric_columns": num_cols, "scaler": method}
    if not num_cols:
        return df, meta

    if method == 'standard':
        scaler = StandardScaler()
    elif method == 'minmax':
        scaler = MinMaxScaler()
    else:
        raise ValueError('Unsupported scaler.')

    df[num_cols] = scaler.fit_transform(df[num_cols])
    return df, meta

## 📥 Ingestão & Visão Rápida

Carregamento dos dados para criar uma visão geral e um relatório de faltantes.  
> Ajuste `INPUT_FILE` e parâmetros do `read_csv` conforme a fonte.


In [ ]:
# Exemplo: ajuste encoding/sep para seu dataset
read_opts = dict(encoding='utf-8', sep=',', low_memory=False)

df = load_csv(INPUT_FILE, **read_opts)
overview = basic_overview(df)
logger.info(json.dumps(overview, indent=2, ensure_ascii=False))
print('Visão geral (resumo):')
print(json.dumps(overview, indent=2, ensure_ascii=False))

missing_df = missing_report(df)
display(missing_df.head(20))

## 🧪 Qualidade & Tipagem

- Remoção de espaços extras em texto.  
- Inferência e **cast** para colunas numéricas que chegaram como string.  
- **Downcast** para reduzir memória.  


In [ ]:
if config['strip_whitespace']:
    df = strip_whitespace(df)

if config['cast_numeric_like']:
    df = infer_numeric_like(df)

if config['infer_types']:
    df = reduce_memory_usage(df)

if config['export_interim']:
    save_parquet(df, OUTPUT_INTERIM)

## 🩹 Tratamento de Valores Faltantes

Estratégias comuns: **simples** (mediana/moda) ou mais avançadas (imputadores específicos por coluna).  
> Mantenha registro do que foi imputado para transparência.


In [ ]:
if config['handle_missing']:
    print('Relatório de faltantes (antes):')
    display(missing_report(df).head(20))

    if config['missing_strategy'] == 'simple':
        df = simple_impute(df)
    else:
        # Espaço para técnicas avançadas personalizadas
        pass

    print('Relatório de faltantes (depois):')
    display(missing_report(df).head(20))

## 🚩 Detecção de Outliers (opcional)

Metodologias: **IQR** (robusta) ou **Z-score**.  
As colunas booleanas `*_is_outlier` são adicionadas para **inspeção** (remover/ajustar é decisão de negócio).


In [ ]:
if config['detect_outliers']:
    if config['outlier_method'] == 'iqr':
        df = detect_outliers_iqr(df)
    elif config['outlier_method'] == 'zscore':
        df = detect_outliers_zscore(df)
    else:
        raise ValueError('Unknown outlier method.')

    outlier_cols = [c for c in df.columns if c.endswith('_is_outlier')]
    logger.info(f'Outlier flags created: {len(outlier_cols)}')

## 🧬 Duplicidades

Remoção de registros idênticos (se fizer sentido).  
> Para duplicidades **parciais** (ex.: chaves), trate caso a caso.


In [ ]:
if config['deduplicate']:
    df = deduplicate_rows(df)

## 🛠️ Engenharia de Atributos (opcional)

Crie atributos úteis ao negócio/modelo: **ratios**, **bins**, **interações** etc.  
> Este bloco é **manual por projeto** — adicione suas transformações aqui.


In [ ]:
if config['feature_engineering']:
    # Exemplo genérico (comente/remova conforme o caso):
    # if {'col_a','col_b'}.issubset(df.columns):
    #     df['a_per_b'] = df['col_a'] / df['col_b'].replace(0, np.nan)
    pass

## 📅 Tratamento de Datas (opcional)

- Conversão para `datetime`.  
- Extração de **ano**, **mês**, **dia**, **dia da semana**, **semana do ano**, **mês textual**, etc.  


In [ ]:
if config['date_features']:
    date_cols = [c for c in df.columns if re.search(r'date|data|dt_', c, re.IGNORECASE)]
    for c in date_cols:
        try:
            df[c] = pd.to_datetime(df[c], errors='coerce', utc=False, infer_datetime_format=True)
        except Exception:
            pass
    # Exemplo de expandir uma coluna de data chamada 'order_date'
    if 'order_date' in df.columns and pd.api.types.is_datetime64_any_dtype(df['order_date']):
        df['order_year'] = df['order_date'].dt.year
        df['order_month'] = df['order_date'].dt.month
        df['order_day'] = df['order_date'].dt.day
        df['order_dow'] = df['order_date'].dt.dayofweek

## 📝 Tratamento de Texto (opcional)

Limpeza, contagens, tamanho de string, presença de termos-chave etc.  
> Use com parcimônia nesta fase; análises específicas podem ir para um notebook próprio.


In [ ]:
if config['text_features']:
    text_cols = [c for c in df.columns if df[c].dtype == 'object']
    for c in text_cols:
        df[f'{c}_len'] = df[c].astype(str).str.len()
        df[f'{c}_word_count'] = df[c].astype(str).str.split().map(len)

## 🔤 Codificação de Categóricas & 🔢 Escalonamento Numérico (opcionais)

**Codificação:** One-hot (seguro) ou Ordinal (compacto).  
**Escala:** Standard (z-score) ou MinMax (0–1) — útil para modelos sensíveis à escala.


In [ ]:
encoding_meta: Dict[str, Any] = {}
scaling_meta: Dict[str, Any] = {}

if config['encode_categoricals']:
    df, encoding_meta = encode_categories(df, encoding=config['encoding_type'])

if config['scale_numeric']:
    df, scaling_meta = scale_numeric(df, method=config['scaler'])

## 💾 Exportação de Artefatos

Salve dados **intermediários** e **prontos** para uso em modelagem/visualização.  
Também salvamos um **manifest** com metadados das transformações.


In [ ]:
manifest: Dict[str, Any] = {
    "created_at": datetime.now().isoformat(timespec='seconds'),
    "random_seed": RANDOM_SEED,
    "config": config,
    "encoding_meta": encoding_meta,
    "scaling_meta": scaling_meta,
    "shape": df.shape,
    "columns": df.columns.tolist()
}

if config['export_interim']:
    save_parquet(df, OUTPUT_INTERIM)

if config['export_processed']:
    save_parquet(df, OUTPUT_PROCESSED)

with open(ARTIFACTS_DIR / 'manifest.json', 'w', encoding='utf-8') as f:
    json.dump(manifest, f, indent=2, ensure_ascii=False)

print('Arquivos gerados:')
print(f'- {OUTPUT_INTERIM if config["export_interim"] else "(pulado)"}')
print(f'- {OUTPUT_PROCESSED if config["export_processed"] else "(pulado)"}')
print(f'- {ARTIFACTS_DIR / "manifest.json"}')

## ✅ Checkpoint & Próximos Passos

- **Revise** as colunas derivadas e decisões (imputação, outliers, codificação).  
- **Documente** no README as escolhas de negócio e justificativas.  
- **Siga** para o próximo notebook (ex.: **N2 — Modelagem** ou **EDA** detalhada).

> Dica: tire **screenshots** de tabelas e distribuições e salve em `prints/` para o portfólio.


## 📎 Apêndice — Anotações Rápidas

Use esta seção como bloco livre para observações específicas do projeto.
